
PROJECT AIM 
-----------
Construct an Hourly Price Forward Curve (HPFC) for electricity markets that:

1. Preserves the observed weekly base price (market consistency)
2. Captures realistic intraday price patterns (peak / off-peak)
3. Incorporates solar production effects 
4. Is stable, interpretable, and suitable for production deployment

MODELING STRATEGY approach:

STAGE 1 - SHAPE LEARNING
* Normalize prices by weekly mean
* Learn pure intraday shapes
* Estimate solar impact on price shape 

STAGE 2 - SCALING
* Reintroduce weekly price level
* Apply weekly scaling
* Ensure weekly mean consistency by construction


In [10]:
# Import modules
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "browser"
from dataclasses import dataclass

# Data Access Layer

In [11]:
class DataLoader:
    def load_spot_prices(self) -> pd.DataFrame:
        """
        replace missing prices with average weekly prices
        """
        df = pd.read_excel('hpfc.xlsx')
        df.columns  = df.columns.str.lower()
        df = df.rename(columns={"datum": "timestamp", "preis": "price"})
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df = df.set_index('timestamp').sort_index()
        full_idx = pd.date_range(start=df.index.min(),
                                end=df.index.max(),
                                freq='h')
        df = df.reindex(full_idx)
        weekly_mean = df['price'].groupby(df.index.to_period('W')).transform('mean')
        df['price'] = df['price'].fillna(weekly_mean)
        df = df.reset_index().rename(columns={"index": "timestamp"})
        return df
    
    def load_solar(self) -> pd.DataFrame:
        """
        Synthesize solar data ---
        """
        rng = pd.date_range(start = "2019-01-01", end = "2023-12-31", freq="h")
        solar = np.clip(np.sin(2 * np.pi * (rng.hour - 6) / 24), 0, 1)   
        df = pd.DataFrame({"timestamp": rng, "solar": solar}) 
        return df
    

Time and Market Structure Helpers

In [12]:
def seasonal_bucket(month: int) -> str:
    """
    Maps months into market-relevant seasonal buckets.
    """
    if month in [5, 6, 7, 8]:
        return "summer"
    if month in [11, 12, 1, 2]:
        return "Winter"
    if month in [10, 3]:
        return "winter_transition"
    return "summer_transition"


def is_peak(ts: pd.Timestamp) -> bool:
    """
    Defines peak hours:
    * Monday - Friday
    * 08:00 - 20:00
    """
    return ts.weekday() < 5 and 8 <= ts.hour <= 20

Shape Model Container

In [13]:
@dataclass
class ShapeModel:
    """
    Stores learned intraday structures.
    """
    x: np.ndarray       # Peak-hour shape (24)
    y: np.ndarray       # Off-peak shape (24)
    beta: dict          # Solar sensitivity per seasonal bucket

# Shape Predictor

In [14]:
class ShapePredictor:
    """
    SHAPE LEARNING
    
    Learns:
    * Peak shape x_k
    * Off-peak shape y_k
    * Solar depression effect beta_b
    
    All predictions are done on WEEKLY-NORMALIZED prices
    to remove absolute price levels.
    """
    
    def fit(self, df: pd.DataFrame) -> ShapeModel:
        df = df.copy()
        
        #   --- Time features ---
        df['week'] = df['timestamp'].dt.isocalendar().week
        df['hour'] = df['timestamp'].dt.hour
        df['bucket'] = df['timestamp'].dt.month.map(seasonal_bucket)
        
        # --- Weekly normalization ---
        df["weekly_mean"] = df.groupby("week")['price'].transform('mean')
        df['s_norm'] = df["price"] / df["weekly_mean"]
        
        # --- Initialize shapes ---
        x = np.zeros(24)
        y = np.zeros(24)
        
        # --- Predict average normalized price per hour ---
        for h in range(24):
            peak_mask = (df['hour'] == h) & df['timestamp'].apply(is_peak)
            off_mask = (df['hour'] == h) & ~df['timestamp'].apply(is_peak)
            
            if peak_mask.any():
                x[h] = df.loc[peak_mask, "s_norm"].mean()
            if off_mask.any():
                y[h] = df.loc[off_mask, "s_norm"].mean()

        # --- Enforce market structure ---
        for h in range(24):
            if not (8 <= h <= 20):
                x[h] = 0
            if 8 <= h <= 20:
                y[h] = 0
                
        # --- Normalize shapes ---
        x /= x.sum()
        y /= y.sum()
        
        # --- Solar sensitivity prediction ---
        beta = {}
        for b in df["bucket"].unique():
            sub = df[df["bucket"] == b]
            if len(sub) > 20:
                # Solar lowers prices => positive beta 
                beta[b] = max(0, np.cov(sub["solar"], sub["s_norm"])[0, 1])
            else:
                beta[b] = 0.0
        
        return ShapeModel(x=x, y=y, beta=beta)          
  

# HPFC Construction 

In [15]:
class HPFCModel:
    """
    PRICE LEVEL REINTRODUCTION
    
    Apply Weekly Scaling:
    * Preserves weekly mean exactly
    * Apply learned shapes and solar correction
    """
    
    def __init__(self, shape_model: ShapeModel):
        self.shape = shape_model
        
    def forecast(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        
        df["hour"] = df["timestamp"].dt.hour
        df["week"] = df["timestamp"].dt.isocalendar().week
        df["bucket"] = df["timestamp"].dt.month.map(seasonal_bucket)
        
        # --- Weekly statistics ---
        df["weekly_mean"] = df.groupby ('week')["price"].transform("mean")
        
        peak_mean = df[df["timestamp"].apply(is_peak)].groupby('week')['price'].mean()
        off_mean = df[~df['timestamp'].apply(is_peak)].groupby('week')['price'].mean()
        
        df['peak_mean'] = df['week'].map(peak_mean)
        df['offpeak_mean'] = df['week'].map(off_mean)
        
        # --- Raw hourly forecast ---
        def raw_price(row):
            h = row['hour']
            price = (
                row["peak_mean"] * self.shape.x[h]
                + row['offpeak_mean'] * self.shape.y[h]
            )
            price -= (
                row['weekly_mean'] * self.shape.beta.get(row['bucket'], 0)
            )
            return price
        
        df['raw_forecast'] = df.apply(raw_price, axis=1)
        
        # --- Weekly scaling ---
        df["raw_week_mean"] = df.groupby('week')["raw_forecast"].transform('mean')
        df['hpfc'] = df['weekly_mean'] * df['raw_forecast'] / df['raw_week_mean']
        return df

# Metrics and Visualization

In [16]:
class HPFCPlots:
    def __init__(self, df, dt_col='timestamp'):
        self.df = df.copy()
        self.df[dt_col] = pd.to_datetime(self.df[dt_col])
        self.df = self.df.set_index(dt_col).sort_index()

    def _layout(self, title):
        return dict(
            title=title,
            hovermode='x unified',
            xaxis=dict(
                tickformat='%a %H:%M',
                dtick=4 * 60 * 60 * 1000,  # 4 hours
                showgrid=True
            ),
            yaxis=dict(title='Price'),
            template='plotly_dark'
        )

    def plot_hpfc_vs_observed(self):
        df = self.df.copy()
        # extract weekday & hour
        df['weekday'] = df.index.weekday   # Mon=0
        df['hour'] = df.index.hour

        # group by weekday + hour
        agg = (
            df
            .groupby(['weekday', 'hour'])[['price', 'hpfc']]
            .mean()
            .reset_index()
        )

        # ordering key: Mon 00 → Sun 23
        agg['week_pos'] = agg['weekday'] * 24 + agg['hour']
        agg = agg.sort_values('week_pos')
        
        # synthetic x-axis only for labeling
        week_start = pd.Timestamp('2025-01-06')  # Monday
        agg['x'] = (
            week_start
            + pd.to_timedelta(agg['weekday'], unit='D')
            + pd.to_timedelta(agg['hour'], unit='H')
        )

        fig = go.Figure()

        fig.add_trace(go.Scatter(
            x=agg['x'],
            y=agg['price'],
            name='price',
            mode='lines'
        ))

        fig.add_trace(go.Scatter(
            x=agg['x'],
            y=agg['hpfc'],
            name='HPFC',
            mode='lines'
        ))

        fig.update_layout(
            title='HPFC vs Observed Prices',
            hovermode='x unified',
            xaxis=dict(
                tickformat='%a %H:%M',
                dtick=4 * 60 * 60 * 1000,
                showgrid=True
            ),
            yaxis=dict(title='Price'),
            template='plotly_dark'
        )
        fig.show(renderer="browser")


# End-to-End Pipeline Execution

In [17]:
# --- Load data ---
loader = DataLoader()
prices = loader.load_spot_prices()
solar = loader.load_solar()

df = prices.merge(solar, on='timestamp')

# --- Predict shape structure ---
estimator = ShapePredictor()
shape_model = estimator.fit(df)

# --- scale to weekly price ---
hpfc = HPFCModel(shape_model)
result = hpfc.forecast(df)

# Weekly structural validation
plots = HPFCPlots(result)
plots.plot_hpfc_vs_observed()

C:\Users\ChamberlinKeuleck\AppData\Local\Temp\ipykernel_24912\4172175086.py:43: FutureWarning:

'H' is deprecated and will be removed in a future version. Please use 'h' instead of 'H'.

